
# Time-Series Comparison Matrix for ET Estimates

This notebook builds pairwise comparison matrices for multiple daily time series contained in a pandas `DataFrame` named **`df`** (datetime index; columns are different ET estimates).

**Outputs**
- Symmetric matrices for: RMSE, MAE, Pearson *r*, Spearman *ρ*, NSE, KGE, DTW (on z-scored series).
- Asymmetric outputs for lead/lag analysis: a matrix of **lag at maximum correlation** (in days) and the corresponding **maximum correlation** value.
- Optional CSV export of each matrix.

**Notes**
- By default, *shape* metrics (correlation, DTW) can be computed on z-scored series (configurable).
- RMSE/MAE/NSE/KGE are computed on raw values (units preserved).
- Missing values are handled pairwise per comparison.
- DTW is O(n²); a Sakoe–Chiba window (e.g., ±30 days) is recommended.


In [ ]:

# --- 1) Imports & configuration ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Matplotlib defaults for clarity
plt.rcParams['figure.figsize'] = (7, 5)
plt.rcParams['figure.dpi'] = 140

# If you want to display more decimals:
pd.options.display.float_format = '{:,.4f}'.format

# ---- USER: ensure `df` is defined in the next cell, or load it from disk ----


In [ ]:

# --- 2) Provide or load your DataFrame `df` here ---------------------------------
# REQUIREMENTS:
# - Daily datetime index (pd.DatetimeIndex) and sorted.
# - Columns are the ET estimates you want to compare.

# EXAMPLE: Uncomment and adapt if you need a quick template for reading a CSV.
# csv_path = "path/to/your_daily_dataframe.csv"
# df = pd.read_csv(csv_path, parse_dates=[0], index_col=0)  # assumes first col is date
# df = df.sort_index()

# If you're working in a notebook where df already exists, you can skip this cell.

# OPTIONAL: Quickly simulate data (comment this block out in production)
# rng = pd.date_range("2023-01-01", "2023-12-31", freq="D")
# rng.name = "date"
# np.random.seed(42)
# df = pd.DataFrame({
#     "ensemble-ET": 3 + np.sin(np.linspace(0, 10*np.pi, len(rng))) + np.random.normal(0, 0.2, len(rng)),
#     "etDis":       3.2 + np.sin(np.linspace(0.1, 10.1*np.pi, len(rng))) + np.random.normal(0, 0.2, len(rng)),
#     "etEns":       2.8 + np.sin(np.linspace(-0.2, 9.8*np.pi, len(rng))) + np.random.normal(0, 0.25, len(rng)),
#     "etPTJ":       3.0 + 0.9*np.sin(np.linspace(0, 10*np.pi, len(rng))) + np.random.normal(0, 0.3, len(rng)),
#     "etSSE":       3.1 + 1.1*np.sin(np.linspace(0.2, 10.2*np.pi, len(rng))) + np.random.normal(0, 0.2, len(rng)),
#     "eteeM":       3.05 + np.sin(np.linspace(0, 10*np.pi, len(rng))) + np.random.normal(0, 0.2, len(rng)),
#     "actual_et_mean": 3.0 + np.sin(np.linspace(0, 10*np.pi, len(rng))) + np.random.normal(0, 0.25, len(rng)),
# }, index=rng)
# df.index.name = "date"

# Optionally restrict to the ET columns of interest:
et_cols = ['ensemble-ET','etDis','etEns','etPTJ','etSSE','eteeM','actual_et_mean']
# et_cols = [c for c in et_cols if c in df.columns]  # uncomment after `df` is defined


In [ ]:

# --- 3) Metric primitives (pairwise-safe) ----------------------------------------

def _pairwise_valid(a, b):
    """Return arrays with NaN/inf removed pairwise."""
    mask = np.isfinite(a) & np.isfinite(b)
    return a[mask], b[mask]

def rmse(a, b):
    a, b = _pairwise_valid(np.asarray(a), np.asarray(b))
    return float(np.sqrt(np.mean((a - b)**2))) if a.size else np.nan

def mae(a, b):
    a, b = _pairwise_valid(np.asarray(a), np.asarray(b))
    return float(np.mean(np.abs(a - b))) if a.size else np.nan

def pearson_r(a, b):
    a, b = _pairwise_valid(np.asarray(a), np.asarray(b))
    if a.size < 2:
        return np.nan
    return float(np.corrcoef(a, b)[0,1])

def spearman_r(a, b):
    a, b = _pairwise_valid(np.asarray(a), np.asarray(b))
    if a.size < 2:
        return np.nan
    return float(stats.spearmanr(a, b, nan_policy='omit').correlation)

def nse(obs, sim):
    o, s = _pairwise_valid(np.asarray(obs), np.asarray(sim))
    if o.size < 2:
        return np.nan
    denom = np.sum((o - np.mean(o))**2)
    if denom == 0:
        return np.nan
    return float(1 - np.sum((o - s)**2) / denom)

def kge(obs, sim):
    o, s = _pairwise_valid(np.asarray(obs), np.asarray(sim))
    if o.size < 2:
        return np.nan
    r = np.corrcoef(o, s)[0,1]
    beta = np.mean(s) / np.mean(o) if np.mean(o) != 0 else np.nan
    cv_o = np.std(o, ddof=1) / np.mean(o) if np.mean(o) != 0 else np.nan
    cv_s = np.std(s, ddof=1) / np.mean(s) if np.mean(s) != 0 else np.nan
    gamma = (cv_s / cv_o) if (cv_o not in [0, np.nan]) else np.nan
    return float(1 - np.sqrt((r-1)**2 + (beta-1)**2 + (gamma-1)**2))

def best_lag_corr(a, b, max_lag=30):
    """Return (lag_in_days, correlation_at_that_lag).
    Positive lag means: b lags a (i.e., a leads).
    Uses pairwise-valid overlap per lag."""
    a = np.asarray(a)
    b = np.asarray(b)
    best_corr = -np.inf
    best_lag = 0
    found = False
    for lag in range(-max_lag, max_lag+1):
        if lag < 0:
            # shift b forward relative to a
            a_slice = a[-lag:]
            b_slice = b[:len(b)+lag]
        elif lag > 0:
            # shift a forward relative to b
            a_slice = a[:len(a)-lag]
            b_slice = b[lag:]
        else:
            a_slice = a
            b_slice = b
        if len(a_slice) < 2:
            continue
        aa, bb = _pairwise_valid(a_slice, b_slice)
        if aa.size < 2:
            continue
        r = np.corrcoef(aa, bb)[0,1]
        if np.isfinite(r) and r > best_corr:
            best_corr = r
            best_lag = lag
            found = True
    if not found:
        return np.nan, np.nan
    return int(best_lag), float(best_corr)

def dtw_distance(x, y, window=None):
    """DTW distance with optional Sakoe–Chiba window (in samples/days for daily data).
    Uses squared Euclidean local cost and returns sqrt of total cost for scale awareness.
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    # Drop pairwise NaNs up-front (strict intersection); conservative but simple.
    xx, yy = _pairwise_valid(x, y)
    n, m = len(xx), len(yy)
    if n == 0 or m == 0:
        return np.nan
    D = np.full((n+1, m+1), np.inf)
    D[0, 0] = 0.0
    for i in range(1, n+1):
        j_min = 1 if window is None else max(1, i - window)
        j_max = m if window is None else min(m, i + window)
        for j in range(j_min, j_max+1):
            cost = (xx[i-1] - yy[j-1])**2
            D[i, j] = cost + min(D[i-1, j], D[i, j-1], D[i-1, j-1])
    return float(np.sqrt(D[n, m]))


In [ ]:

# --- 4) Driver: build comparison matrices ----------------------------------------

def compare_series(df, columns, max_lag=30, dtw_window=30, z_for_shape=True, save_csv=False, out_dir=None):
    """Compute pairwise comparison matrices for selected columns in df.

    Parameters
    ----------
    df : pandas.DataFrame
        Daily time series with a DatetimeIndex; columns are ET estimates.
    columns : list[str]
        Subset of df columns to compare.
    max_lag : int, default 30
        Max absolute lag (in days) to search for best-lag correlation.
    dtw_window : int or None, default 30
        Sakoe–Chiba window (±window) for DTW in days/samples. Use None for unconstrained.
    z_for_shape : bool, default True
        If True, compute Pearson/Spearman/DTW on z-scored versions (shape-focused).
    save_csv : bool, default False
        If True, saves each matrix as CSV in out_dir.
    out_dir : str or Path, optional
        Destination directory for CSVs if save_csv is True.

    Returns
    -------
    dict[str, pandas.DataFrame]
        A dictionary of matrices keyed by metric names.
    """
    df = df.copy()
    df = df.sort_index()
    # Ensure daily frequency (doesn't fill gaps; assumes input is already daily or nearly so)
    # If you want strict daily alignment with interpolation, do it explicitly before calling.

    cols = [c for c in columns if c in df.columns]
    if len(cols) < 2:
        raise ValueError("Need at least two valid columns to compare.")
    X = df[cols]

    # Z-score version for shape metrics if requested
    if z_for_shape:
        Xz = (X - X.mean()) / X.std(ddof=1)
    else:
        Xz = X.copy()

    n = len(cols)
    idx = pd.Index(cols, name="series")

    # Initialize matrices
    RMSE   = pd.DataFrame(np.nan, index=idx, columns=idx)
    MAE    = pd.DataFrame(np.nan, index=idx, columns=idx)
    PEAR   = pd.DataFrame(np.nan, index=idx, columns=idx)  # Pearson r
    SPEAR  = pd.DataFrame(np.nan, index=idx, columns=idx)  # Spearman rho
    NSEm   = pd.DataFrame(np.nan, index=idx, columns=idx)
    KGEm   = pd.DataFrame(np.nan, index=idx, columns=idx)
    LAG    = pd.DataFrame(np.nan, index=idx, columns=idx)  # best lag (days), sign: col_j lags col_i if +lag
    R_AT_LAG = pd.DataFrame(np.nan, index=idx, columns=idx)  # max correlation at that lag
    DTWm   = pd.DataFrame(np.nan, index=idx, columns=idx)  # DTW on z-scored series if z_for_shape

    # Compute pairwise
    for i, ci in enumerate(cols):
        for j, cj in enumerate(cols):
            if j < i:
                # fill symmetry from previously computed value where applicable
                RMSE.loc[ci, cj] = RMSE.loc[cj, ci]
                MAE.loc[ci, cj]  = MAE.loc[cj, ci]
                PEAR.loc[ci, cj] = PEAR.loc[cj, ci]
                SPEAR.loc[ci, cj]= SPEAR.loc[cj, ci]
                NSEm.loc[ci, cj] = NSEm.loc[cj, ci]
                KGEm.loc[ci, cj] = KGEm.loc[cj, ci]
                DTWm.loc[ci, cj] = DTWm.loc[cj, ci]
                # Asymmetric metrics (lag) are not symmetric; compute both directions
                continue

            xi = X[ci].to_numpy()
            xj = X[cj].to_numpy()
            xi_z = Xz[ci].to_numpy()
            xj_z = Xz[cj].to_numpy()

            # Units-based (raw)
            RMSE.loc[ci, cj] = rmse(xi, xj)
            MAE.loc[ci, cj]  = mae(xi, xj)
            NSEm.loc[ci, cj] = nse(xi, xj)     # treat ci as "obs", cj as "sim"
            KGEm.loc[ci, cj] = kge(xi, xj)     # same convention

            # Shape-based (z if requested)
            PEAR.loc[ci, cj]  = pearson_r(xi_z, xj_z)
            SPEAR.loc[ci, cj] = spearman_r(xi_z, xj_z)

            # Best lag & correlation (computed on z-scored for stability)
            lag, rmax = best_lag_corr(xi_z, xj_z, max_lag=max_lag)
            LAG.loc[ci, cj] = lag
            R_AT_LAG.loc[ci, cj] = rmax

            # DTW (z-scored)
            DTWm.loc[ci, cj] = dtw_distance(xi_z, xj_z, window=dtw_window)

    # Mirror symmetric parts
    RMSE = RMSE.combine_first(RMSE.T)
    MAE  = MAE.combine_first(MAE.T)
    PEAR = PEAR.combine_first(PEAR.T)
    SPEAR= SPEAR.combine_first(SPEAR.T)
    NSEm = NSEm.combine_first(NSEm.T)
    KGEm = KGEm.combine_first(KGEm.T)
    DTWm = DTWm.combine_first(DTWm.T)

    matrices = {
        "RMSE": RMSE, "MAE": MAE, "Pearson_r(z)": PEAR, "Spearman_rho(z)": SPEAR,
        "NSE(ci as obs, cj as sim)": NSEm,
        "KGE(ci as obs, cj as sim)": KGEm,
        "Lag_at_max_r(z) [days]": LAG,
        "Max_r_at_lag(z)": R_AT_LAG,
        "DTW_distance(z)": DTWm,
    }

    if save_csv:
        import os
        out_dir = os.fspath(out_dir or ".")
        os.makedirs(out_dir, exist_ok=True)
        for name, M in matrices.items():
            M.to_csv(os.path.join(out_dir, f"{name.replace(' ', '_').replace('/', '_')}.csv"))

    return matrices


In [ ]:

# --- 5) Example usage -------------------------------------------------------------
# Ensure `df` and `et_cols` exist (see earlier cell). Then run:
#
# matrices = compare_series(
#     df=df,
#     columns=et_cols,
#     max_lag=30,       # days to search for best lag
#     dtw_window=30,    # DTW Sakoe–Chiba window (None for unconstrained; slower)
#     z_for_shape=True, # z-score for correlation & DTW
#     save_csv=False,   # set True to write CSVs
#     out_dir="./et_matrices"
# )
#
# # Access individual matrices:
# matrices["RMSE"]
# matrices["Pearson_r(z)"]
# matrices["Lag_at_max_r(z) [days]"]
# matrices["DTW_distance(z)"]
#
# # Display as heatmaps (one figure per matrix):
# for name, M in matrices.items():
#     plt.figure()
#     plt.imshow(M.values, origin='upper', interpolation='nearest')
#     plt.title(name)
#     plt.xticks(range(len(M.columns)), M.columns, rotation=45, ha='right')
#     plt.yticks(range(len(M.index)), M.index)
#     plt.colorbar()
#     plt.tight_layout()
#     plt.show()
